<a href="https://colab.research.google.com/github/cwerries/IANNWTF_Group_14_Submissions/blob/master/text_processing_incomplete.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
!pip install -q tensorflow-text
import matplotlib.pyplot as plt
import numpy as np
import re
import string

import tensorflow as tf
from keras.preprocessing.sequence import skipgrams 
from keras.preprocessing import text
import tensorflow_text as tf_text
import tensorflow_datasets as tfds
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Dropout, Conv2D, Flatten, MaxPooling2D, Conv2DTranspose, Reshape, Activation, BatchNormalization, GlobalAvgPool2D

from keras.layers.core import Dense, Reshape
from keras.layers.embeddings import Embedding
from keras.models import Model,Sequential

In [3]:
import os
from google.colab import drive
drive._mount('/content/drive')
os.chdir('drive/MyDrive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Preprocessing

Get most_common_words

In [5]:
def most_common_words(sentences): 
  tokenized = list(sum(sentences, []))
  count = {}
  for w in tokenized:
    count[w] = count.get(w, 0) + 1  # create dictionary of words with counts

  sorted_counts = dict(sorted(count.items(), key=lambda item: item[1], reverse=True))  #sort dictionary counts
  #print(sorted_counts)
  keys = [key for key in sorted_counts.keys()][:100]   #get only the words -> here 10 most common words

  return keys

In [75]:
with open('bible.txt') as f:
    file = f.readlines()

file = file[:100]

sentences = []
for sentence in file:
  if sentence != "\n":
    #sentence = sentence.rstrip("\n")
    sentence = sentence.translate(str.maketrans('', '', string.digits))
    sentence = sentence.translate(str.maketrans('', '', string.punctuation))
    sentences.append(sentence.lower().split())
print("sentences: ", sentences[:2])

keys = most_common_words(sentences)
print("keys: ", keys[:5])

# all_tokens = list(sum(sentences, []))
# print("all_tokens: ", all_tokens[:100])
# most_common_tokens = [token for token in all_tokens if token in keys]
# print("most_common_tokens: ", most_common_tokens[:100])

# sentences with only most common tokens
norm_bible = [[token for token in s if token in keys] for s in sentences]
print("norm_bible: ", norm_bible[:2])

norm_bible = [' '.join(tok_sent) for tok_sent in sentences]

sentences:  [['the', 'first', 'book', 'of', 'moses', 'called', 'genesis'], ['in', 'the', 'beginning', 'god', 'created', 'the', 'heaven', 'and', 'the', 'earth']]
keys:  ['the', 'and', 'god', 'of', 'earth']
norm_bible:  [['the', 'first', 'book', 'of', 'moses', 'called', 'genesis'], ['in', 'the', 'god', 'created', 'the', 'heaven', 'and', 'the', 'earth']]


assigning numeric identifier

In [76]:
tokenizer = text.Tokenizer()
tokenizer.fit_on_texts(norm_bible)
word_id = tokenizer.word_index

id_word = {id:word for word, id in word_id.items()} #first index then word

vocab_size = len(word_id) + 1 # padding included
word_ids = [[word_id[id] for id in text.text_to_word_sequence(sent)] for sent in norm_bible]
print('Vocabulary Size: ', vocab_size)
print('Vocabulary Sample: ', list(word_id.items())[:5]) 
print("word_ids: ", word_ids[:2])

Vocabulary Size:  151
Vocabulary Sample:  [('the', 1), ('and', 2), ('god', 3), ('of', 4), ('earth', 5)]
word_ids:  [[1, 65, 98, 4, 99, 28, 100], [8, 1, 101, 3, 34, 1, 25, 2, 1, 5]]


In [72]:
# generate skip-grams
skip_grams = [skipgrams(w_id, vocabulary_size=vocab_size, window_size=4, negative_samples = 0.0) for w_id in word_ids]

# view sample skip-grams
pairs, labels = skip_grams[0][0], skip_grams[0][1]

for i in range(5):
    print("({:s} ({:d}), {:s} ({:d}))".format(
          id_word[pairs[i][0]], pairs[i][0], 
          id_word[pairs[i][1]], pairs[i][1])) 

(book (98), genesis (100))
(genesis (100), of (4))
(the (1), moses (99))
(first (65), book (98))
(of (4), the (1))


List of Skipgrams with word ids

In [61]:
# keep only wird ids
list_pairs = []
for pair, _ in skip_grams:
  list_pairs.append(pair)

#flatten list
flat_list = []
for sublist in list_pairs:
  for item in sublist:
    flat_list.append(item)

word_pairs_array = np.array(flat_list)
print(word_pairs_array[:3])

[[100  98]
 [ 99  98]
 [  1   4]]


Create tensor dataset

In [64]:
word_pairs_tensor =tf.data.Dataset.from_tensor_slices((word_pairs_array[:,0], word_pairs_array[:,1])) # create our data_set with inputs and targets
prepared_ds = word_pairs_tensor.shuffle(1000).batch(32).prefetch(32)

# Model

In [69]:
class SkipGram(tf.keras.layers.Layer):
    def __init__(self, vocab_size, embedding_size):
        super(SkipGram, self).__init__()
        self.vocab_size = vocab_size
        self.embedding_size = embedding_size

        self.build()

    def build(self, input_shape):
      self.emb_matrix = self.add_weight(
          shape=(self.vocab_size, self.embedding_size),
          initializer='GlorotNormal'
          )
      self.score_matrix = self.add_weight(
          shape=(self.vocab_size, self.embedding_size),
          initializer='GlorotNormal'
          )
      self.score_bias = self.add_weight(
          shape=(self.vocab_size),
          initializer='zeros'
          )

    def call(self, input, labels):
        embeddings = tf.nn.embedding_lookup(self.emb_matrix,inputs)
        nce_loss = tf.nn.nce_loss(
            weights = self.score_matrix,
            biases = self.score_bias,
            labels = labels,
            inputs = embeddings,
            num_sampled = 15,
            num_classes = self.vocab
            )
        loss = tf.reduce_mean(nce_loss)
        return loss

# Training

In [70]:
def nearest_neighbours(model, k):
  embeddings = 
    for word in check_words:
        word_embedding = 
        cosine_sim = -dot(embeddings, word_embedding)/(norm(embeddings)*norm(word_embedding))
        nearest_neighbours = np.argsort(cosine_sim)[:k]
        print("to Word ", word," the nearest neighbours are",.join([id2word[n] for n in nearest_neighbours )

SyntaxError: ignored

In [71]:
tf.keras.backend.clear_session()

# define hyperparams
learning_rate = 0.01
num_epochs = 10
embedding_size = 64
k = 5

model = SkipGram(vocab_size, embedding_size)
optimizer = tf.optimizers.Adam(learning_rate)
check_words= ["holy", "father", "wine", "poison", "love", "strong", "day" ]

losses = []

# We train for num_epochs epochs.
for epoch in range(num_epochs):
    print(f'Epoch {str(epoch)}')

    #training
    epoch_loss_agg = []
    for input,target in prepared_ds:

      with tf.GradientTape() as tape:
        loss = model(input, target) # get the loss
      gradients = tape.gradient(loss, model.trainable_variables)
      optimizer.apply_gradients(zip(gradients, model.trainable_variables))
      epoch_loss_agg.append(loss)
                   
    losses.append(tf.reduce_mean(epoch_loss_agg))
    
    print(f'Epoch: {str(epoch)} with loss {losses[-1]}')

    # after each epoch k nearest words
    nearest_neighbours(model, k)


TypeError: ignored